# Projeto: Modelo Gerador de Resumo-Noticia de Partidas de Futebol

## 🎯 Objetivo

Este projeto tem como objetivo desenvolver um modelo capaz de gerar uma narrativa jornalística neutra e personalizada sobre uma partida de futebol, com base nos eventos ocorridos minuto a minuto. O modelo será treinado para identificar momentos chave e outras informações relevantes durante o jogo e, em seguida, produzir uma manchete acompanhada de um resumo no estilo jornalístico.

## 🧩 Funcionamento

Primeiramente, foram coletados dados a partir da [basedosdados](https://basedosdados.org/dataset/c861330e-bca2-474d-9073-bc70744a1b23?table=18835b0d-233e-4857-b454-1fa34a81b4fa&utm_term=estat%C3%ADstica%20de%20futebol&utm_campaign=Conjuntos+de+Dados+-+BD+Pro&utm_source=adwords&utm_medium=ppc&hsa_acc=9488864076&hsa_cam=20477806005&hsa_grp=159509357264&hsa_ad=670893357725&hsa_src=g&hsa_tgt=kwd-354220152183&hsa_kw=estat%C3%ADstica%20de%20futebol&hsa_mt=b&hsa_net=adwords&hsa_ver=3&gad_source=1&gad_campaignid=20477806005&gbraid=0AAAAApsIj8wlxMNuoLNEgfS-0UJinMLQ-&gclid=Cj0KCQjw8cHABhC-ARIsAJnY12zrme3TgSNT3gAf23eUfvyue0--_RdOX5nTYFL4o_yXCNejVGxI97YaAh0aEALw_wcB), com informações das datas dos jogos e os times que irão se enfrentar ao longo de uma temporada.

Diante disso, a coleta dos lances é feita via web scraping utilizando as bibliotecas `requests` e `BeautifulSoup`, a partir da plataforma [Lance!](https://www.lance.com.br/temporeal/agenda/abril-2025), que transmite lances em tempo real. O processo consiste em:
  1. Coleta de dados de uma temporada na basedosdados;
  2. Scraping de lances de partidas daquela temporada

    - Consultas na plataforma Lance! possuem a seguinte estrutura:
      - https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-[ano]-[dia]-[mes]-[ano]-[time_mandante]x[time_visitante]
      - por exemplo: https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2025-26-04-2025-mirassolxatletico-mg
  3. Estruturação do dataset com notícias referentes a cada partida

Em todas as coletas, os dados são padronizados.

In [1]:
!pip install basedosdados

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.1 MB/s eta 0:00:00


In [2]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata

In [3]:
# Times da Série A 2023
times_padronizados_2023 = {
    "america-mg", "athletico-pr", "atletico-mg", "bahia", "botafogo",
    "red-bull-bragantino", "corinthians", "coritiba", "cruzeiro", "cuiaba",
    "flamengo", "fluminense", "fortaleza", "goias", "gremio",
    "internacional", "palmeiras", "santos", "sao-paulo", "vasco"
}

In [4]:
# Abaixo são definidas funções para padronização de nomes, url e lances.
# Necessário tanto para estruturação do dataset, quanto para o scraping.

# Função para padronizar nome dos times
def padronizar_nome(nome):
    # Remove acentuação
    nome = unicodedata.normalize('NFD', nome)
    nome = ''.join(c for c in nome if unicodedata.category(c) != 'Mn')

    nome = nome.lower()
    nome = nome.replace(" ", "-")
    nome = nome.replace("ec-", "").replace("ec ", "")
    nome = nome.replace("rb-bragantino", "red-bull-bragantino")
    nome = nome.replace("cuiaba-mt", "cuiaba")  # já sem acento
    nome = nome.replace("vasco-da-gama", "vasco")
    nome = nome.replace("ec-vitoria", "vitoria")
    nome = nome.replace("ec-bahia", "bahia")
    nome = nome.replace("coritiba-fc", "coritiba")
    return nome


# Função para montar a URL
def montar_url(data, mandante, visitante):
    data_formatada = data.strftime("%d-%m-%Y")
    return f"https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2023-{data_formatada}-{mandante}x{visitante}"

# Função para tentar extrair os lances
def extrair_lances(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None
        soup = BeautifulSoup(response.content, "html.parser")
        lances = []
        for li in soup.select("ul.pl-11 li"):
            texto = li.select_one("span.break-words")
            if texto:
                lances.append(texto.get_text(strip=True))
        return lances
    except Exception as e:
        print(f"Erro no scraping de {url}: {e}")
        return None

In [5]:
# Coleta de partidas e datas da temporada na basedosdados.

import basedosdados as bd

billing_id = "pln-2025-1"

# Faz a consulta dos jogos
query = """
SELECT
  ano_campeonato,
  data,
  time_mandante,
  time_visitante
FROM `basedosdados.mundo_transfermarkt_competicoes.brasileirao_serie_a`
WHERE ano_campeonato = 2023
"""

df = bd.read_sql(query=query, billing_project_id=billing_id)

# Corrige nomes de times
df["time_mandante"] = df["time_mandante"].apply(padronizar_nome)
df["time_visitante"] = df["time_visitante"].apply(padronizar_nome)


Downloading: 100%|██████████|


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ano_campeonato  380 non-null    Int64 
 1   data            380 non-null    dbdate
 2   time_mandante   380 non-null    object
 3   time_visitante  380 non-null    object
dtypes: Int64(1), dbdate(1), object(2)
memory usage: 12.4+ KB


In [7]:
df.head()

,ano_campeonato,data,time_mandante,time_visitante
0,2023,2023-04-22,fluminense,athletico-pr
1,2023,2023-05-29,botafogo,america-mg
2,2023,2023-07-16,internacional,palmeiras
3,2023,2023-07-16,athletico-pr,bahia
4,2023,2023-10-19,goias,sao-paulo


In [8]:
import time
from datetime import datetime, timedelta

# Lista para armazenar os dados
dados = []

for idx, row in df.iterrows():
    data_original = row["data"]
    mandante = row["time_mandante"]
    visitante = row["time_visitante"]

    tentativas = [0, -1, 1]  # tenta no dia, um dia antes e um dia depois
                             # necessário pois algumas datas estavam diferentes
                             # entre a basedosdados e a da noticia
                             # por uma dif de um dia

    sucesso = False
    for delta in tentativas:
        data_tentativa = data_original + timedelta(days=delta)
        url = montar_url(data_tentativa, mandante, visitante)
        print(f"Coletando: {url}")

        lances = extrair_lances(url)
        if lances:
            dados.append({
                "data": data_tentativa.strftime("%Y-%m-%d"),
                "mandante": mandante,
                "visitante": visitante,
                "url": url,
                "lances": lances
            })
            sucesso = True
            break
        else:
            print(f"Falha em {url} (tentativa delta {delta})")

    if not sucesso:
        print(f"⚠️ Não foi possível encontrar lances para {mandante} x {visitante}")

    time.sleep(1)  # Espera para não sobrecarregar o site

Coletando: https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2023-22-04-2023-fluminensexathletico-pr
Coletando: https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2023-29-05-2023-botafogoxamerica-mg
Falha em https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2023-29-05-2023-botafogoxamerica-mg (tentativa delta 0)
Coletando: https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2023-28-05-2023-botafogoxamerica-mg
Coletando: https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2023-16-07-2023-internacionalxpalmeiras
Coletando: https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2023-16-07-2023-athletico-prxbahia
Coletando: https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2023-19-10-2023-goiasxsao-paulo
Falha em https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2023-19-10-2023-goiasxsao-paulo (tentativa delta 0)
Coletando: https://www.lance.com.br/temporeal/partida/brasileiro-serie-a-2023-18-10-2023-goia

In [9]:
# Converte para DataFrame final
df_lances = pd.DataFrame(dados)

df_lances.head()

,data,mandante,visitante,url,lances
0,2023-04-22,fluminense,athletico-pr,https://www.lance.com.br/temporeal/partida/bra...,[APITA O ÁRBITRO! FINAL DE PARTIDA! O Fluminen...
1,2023-05-28,botafogo,america-mg,https://www.lance.com.br/temporeal/partida/bra...,[APITA O ÁRBITRO! FINAL DE PARTIDA! O Botafogo...
2,2023-07-16,internacional,palmeiras,https://www.lance.com.br/temporeal/partida/bra...,[NÃO HÁ TEMPO PARA MAIS NADA! FIM DE JOGO! Int...
3,2023-07-16,athletico-pr,bahia,https://www.lance.com.br/temporeal/partida/bra...,[TERMINA O JOGO!! ATHLETICO JOGA BEM NO PRIMEI...
4,2023-10-18,goias,sao-paulo,https://www.lance.com.br/temporeal/partida/bra...,[ACABOU! NÃO HÁ TEMPO PARA MAIS NADA! GOIÁS VE...


In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Função para extrair os lances formatados com minuto + texto
def extrair_lances_formatados(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None

        soup = BeautifulSoup(response.text, 'html.parser')
        ul = soup.find('ul', class_='pl-11')

        if not ul:
            return None

        lances = []
        for li in ul.find_all('li', class_='block'):
            minuto_span = li.find('span', class_='bg-white')
            texto_span = li.find('span', class_='break-words')

            if minuto_span and texto_span:
                minuto = minuto_span.get_text(strip=True).replace("''", "'")
                texto = texto_span.get_text(strip=True)
                lance_formatado = f"[{minuto}] {texto}"
                lances.append(lance_formatado)

        return lances
    except Exception as e:
        print(f"Erro ao extrair lances de {url}: {e}")
        return None

# Agora criar o novo DataFrame
partidas = []

for partida in dados:
    url = partida["url"]
    lances = extrair_lances_formatados(url)

    if lances:
        texto_lances = " ".join(lances)  # Concatena todos os lances em um único texto
    else:
        texto_lances = ""

    partidas.append({
        "data": partida["data"],
        "mandante": partida["mandante"],
        "visitante": partida["visitante"],
        "lances_concatenados": texto_lances,
        "url": partida["url"]
    })

# Cria o DataFrame final
df_lances = pd.DataFrame(partidas)

# Exibe uma prévia
df_lances.head()


,data,mandante,visitante,lances_concatenados,url
0,2023-04-22,fluminense,athletico-pr,[52'] APITA O ÁRBITRO! FINAL DE PARTIDA! O Flu...,https://www.lance.com.br/temporeal/partida/bra...
1,2023-05-28,botafogo,america-mg,[50'] APITA O ÁRBITRO! FINAL DE PARTIDA! O Bot...,https://www.lance.com.br/temporeal/partida/bra...
2,2023-07-16,internacional,palmeiras,[50'] NÃO HÁ TEMPO PARA MAIS NADA! FIM DE JOGO...,https://www.lance.com.br/temporeal/partida/bra...
3,2023-07-16,athletico-pr,bahia,[50'] TERMINA O JOGO!! ATHLETICO JOGA BEM NO P...,https://www.lance.com.br/temporeal/partida/bra...
4,2023-10-18,goias,sao-paulo,[52'] OLÉ! Final da partida é marcado por grit...,https://www.lance.com.br/temporeal/partida/bra...


In [12]:
# Separar e mostrar os lances do índice 0
lances_0 = df_lances['lances_concatenados'].iloc[0].split('] ')  # Divide os lances pela marcação do minuto
lances_0 = [lance.strip() for lance in lances_0 if lance]  # Limpa qualquer item vazio e formata

# Exibe os lances detalhados
for lance in lances_0:
    print(lance)


[52'
APITA O ÁRBITRO! FINAL DE PARTIDA! O Fluminense vence bem em casa a equipe do Athletico-PR por 2x0 e mantém os 100% pela segunda rodada do Brasileirão! [51'
Assistências para finalizações: Fluminense 6, Athletico-PR 7 [50'
UHH!!! Vítor Bueno bateu a falta na faixa central mandando na direção da meta e a bola passa muito perto do travessão perigosamente [49'
AMARELADO! Thiago Santos chega empurrando Vítor Roque pelas costas e Paulo Zanovelli aplica o amarelo para o volante [48'
Finalizações até aqui: Fluminense 13, Athletico-PR 11 [47'
Jhon Arias passa com Samuel Xavier na área que devolve rápido com o camisa 21 e a bandeira já estava sinalizando o fora de jogo do lateral do Flu [46'
NA REDE PELO LADO DE FORA!!! Vítor Bueno deu uma bela fatiada na bola da esquerda achando a entrada pela diagonal na grande área de Madson e o lateral finaliza no canto esquerdo de Fábio para fora do gol [45'
+7! Vamos até os 52 minutos de partida [43'
Alexsander engatilhou o chute de longe em seu seto

In [13]:
df_lances.to_csv("lances_brasileirao_2023.csv", index=False, encoding='utf-8')

In [16]:
import pandas as pd

# Caminho para o arquivo CSV
path = '/content/lances_brasileirao_2023 (1).csv'

# Lê o CSV e cria o DataFrame
df = pd.read_csv(path)

# Exibe as primeiras linhas do DataFrame
df.head()


,data,mandante,visitante,lances_concatenados,url,resumo
0,2023-04-22,fluminense,athletico-pr,[52'] APITA O ÁRBITRO! FINAL DE PARTIDA! O Flu...,https://www.lance.com.br/temporeal/partida/bra...,O Fluminense começou com uma alta intensidade ...
1,2023-05-28,botafogo,america-mg,[50'] APITA O ÁRBITRO! FINAL DE PARTIDA! O Bot...,https://www.lance.com.br/temporeal/partida/bra...,Botafogo vence América-MG em casa e amplia van...
2,2023-07-16,internacional,palmeiras,[50'] NÃO HÁ TEMPO PARA MAIS NADA! FIM DE JOGO...,https://www.lance.com.br/temporeal/partida/bra...,Inter e Palmeiras decepcionam e empatam sem go...
3,2023-07-16,athletico-pr,bahia,[50'] TERMINA O JOGO!! ATHLETICO JOGA BEM NO P...,https://www.lance.com.br/temporeal/partida/bra...,"Athletico vence o Bahia, se recupera e entra n..."
4,2023-10-18,goias,sao-paulo,[52'] OLÉ! Final da partida é marcado por grit...,https://www.lance.com.br/temporeal/partida/bra...,"Goiás vence, aumenta jejum do São Paulo, mas n..."


## ⚠️ Observações

- No dataset acima, os resumos foram adicionados manualmente e nem todos foram adicionados. Atualmente, está em processo de desenvolvimento um scrapper para os resumos. Dos que foram adicionados, a maioria são do site do ge.globo.com ou da própria plataforma Lance!. O objetivo é adicionar mais partidas de outras temporadas e notícias para melhorar a qualidade de resposta do modelo.

- T5 (Text-to-Text Transfer Transformer) e BERT (Bidirectional Encoder Representations from Transformers) são os dois canditados em análise para produção do modelo final.